In [ ]:
import boto3
from IPython.display import Image as IImage, display
from IPython.display import HTML, display
from PIL import Image, ImageDraw, ImageFont
import time
import os

In [ ]:
rekognition = boto3.client('rekognition')
s3 = boto3.client('s3')

In [ ]:
bucketName = "{REPLACE_YOUR_BUCKET}"
imageName = "media/shots_video.mp4"

In [ ]:
s3.upload_file(
    Filename="resources/shots_video.mp4",
    Bucket=bucketName,
    Key=imageName,
)

In [ ]:

videoName = "media/shots_video.mp4"
s3VideoUrl = s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': videoName})
videoTag = "<video controls='controls' width='640' height='360' name='Video' src='{0}'></video>".format(s3VideoUrl)
videoui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(videoTag)
display(HTML(videoui))

In [ ]:
startSegmentDetection = rekognition.start_segment_detection(
    Video={
        'S3Object': {
            'Bucket': bucketName,
            'Name': videoName,
        },
    },
    SegmentTypes=['SHOT']
)

segmentationJobId = startSegmentDetection['JobId']
display("Job Id: {0}".format(startSegmentDetection))

In [ ]:
getSegmentDetection = rekognition.get_segment_detection(
    JobId=segmentationJobId
)

while(getSegmentDetection['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    getSegmentDetection = rekognition.get_segment_detection(
    JobId=segmentationJobId)
    
display(getSegmentDetection['JobStatus'])

In [ ]:
print(getSegmentDetection)

In [ ]:
for technicalCue in getSegmentDetection['Segments']:
    print(technicalCue)
    
    frameStartValue = technicalCue['StartTimestampMillis']
    #Divide by 1000 to convert from milliseconds to seconds
    frameStartValue = frameStartValue/1000.0
    
    frameEndValue = technicalCue['EndTimestampMillis']
    #Divide by 1000 to convert from milliseconds to seconds
    frameEndValue = frameEndValue/1000.0
    
    videoTag = "<video width='640' controls loop height='360' name='Video' src='{0}{1}{2}{3}{4}'></video>".format(s3VideoUrl,'#t=',frameStartValue,',',frameEndValue)
    videoui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(videoTag)
    display(HTML(videoui))